In [ ]:
import polars

data = polars.read_csv("../data/processed/simplified_coffee.csv")

data.head()

name,roaster,roast,loc_country,origin,100g_USD,rating,review_date,review
str,str,str,str,str,f64,i64,str,str
"""Ethiopia Shaki…","""Revel Coffee""","""Medium-Light""","""United States""","""Ethiopia""",4.7,92,"""November 2017""","""Crisply sweet,…"
"""Ethiopia Suke …","""Roast House""","""Medium-Light""","""United States""","""Ethiopia""",4.19,92,"""November 2017""","""Delicate, swee…"
"""Ethiopia Gedeb…","""Big Creek Coff…","""Medium""","""United States""","""Ethiopia""",4.85,94,"""November 2017""","""Deeply sweet, …"
"""Ethiopia Kayon…","""Red Rooster Co…","""Light""","""United States""","""Ethiopia""",5.14,93,"""November 2017""","""Delicate, rich…"
"""Ethiopia Gelge…","""Willoughby's C…","""Medium-Light""","""United States""","""Ethiopia""",3.97,93,"""November 2017""","""High-toned, fl…"


In [36]:
from pathlib import Path
from llama_index import Document
documents = []
for coffee_review in data.rows(named=True):
    meta = {"Name": coffee_review["name"], "roaster": coffee_review["roaster"]}
    doc = Document(text=coffee_review["review"], metadata=meta)
    documents.append(doc)

print(f"Made {len(documents)} documents")


Made 1246 documents


In [37]:
from llama_index import VectorStoreIndex
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import Ollama as OllamaLLM
from llama_index.embeddings import OllamaEmbedding

llm = OllamaLLM(model="mistral", request_timeout=30.0)
embed = OllamaEmbedding("phi")
context = ServiceContext.from_defaults(llm=llm, embed_model=embed)


In [38]:
index = VectorStoreIndex.from_documents(documents, service_context=context, show_progress=True)

Parsing nodes:   0%|          | 0/1246 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1246 [00:00<?, ?it/s]

In [41]:
query = index.as_query_engine()
resp = query.query("I hate coffee. Which coffee would change my mind?")
print(resp)

 Based on the descriptions provided, the Ethiopia Yirgacheffe G1 from VERYTIME could be a good option to consider if you're open to trying coffees with a natural processing method. This coffee is described as having a crisp sweetness and deep chocolaty flavors with hints of spicy florals, which might appeal to those who enjoy sweeter or chocolatey flavors. If you prefer a more acidic or fruit-forward profile, the Cate Natural G5 from Small Eyes Cafe could also be worth trying due to its bright and tart characteristics with notes of raspberry, chocolate fudge, sandalwood, narcissus, and almond butter. Ultimately, everyone's taste preferences are unique, so it may take some experimentation to find a coffee that resonates with you.
